In [7]:
import pandas as pd 

customers  = pd.read_csv("customers.csv", parse_dates=["join_date"])
daily_sales = pd.read_csv("daily_sales.csv", parse_dates=["date"])
orders = pd.read_csv("orders.csv", parse_dates=["order_date"])
stores = pd.read_csv("stores.csv")

for name, df in {
    "cutomers" : customers,
    "daily_sales": daily_sales, 
    "orders": orders, 
    "stores" : stores
}.items():
    print(f"\n=== {name} ===")
    print(df.head())
    print("shape:", df.shape)
    print("missing:\n", df.isna().sum())


=== cutomers ===
   cust_id  join_date region
0     1000 2023-02-06     FL
1     1001 2023-03-07     NY
2     1002 2023-05-14     CA
3     1003 2023-04-12     TX
4     1004 2023-03-03     CA
shape: (220, 3)
missing:
 cust_id      0
join_date    0
region       0
dtype: int64

=== daily_sales ===
   store_id       date   amount  discount region quarter
0       100 2024-01-01  2492.85      0.05     NY  2024Q1
1       100 2024-01-02  1282.18      0.09     NY  2024Q1
2       100 2024-01-04  1713.75      0.11     NY  2024Q1
3       100 2024-01-06  1020.63      0.01     NY  2024Q1
4       100 2024-01-07  1315.77      0.11     NY  2024Q1
shape: (3150, 6)
missing:
 store_id    0
date        0
amount      0
discount    0
region      0
quarter     0
dtype: int64

=== orders ===
   order_id  cust_id  store_id order_date  amount  discount
0         1     1194       103 2024-12-27  293.63      0.07
1         2     1172       104 2024-01-14  135.05      0.08
2         3     1140       105 2024-01-12

In [8]:
customers["cust_id"] = customers["cust_id"].astype("int64")
stores["store_id"]   = stores["store_id"].astype("int64")
orders["store_id"]   = orders["store_id"].astype("int64")
orders["cust_id"]    = orders["cust_id"].astype("int64")
orders["discount"] = orders["discount"].fillna(0)
orders_clean = orders.dropna(subset=["amount"]).copy()
print(orders_clean)


     order_id  cust_id  store_id order_date  amount  discount
0           1     1194       103 2024-12-27  293.63      0.07
1           2     1172       104 2024-01-14  135.05      0.08
2           3     1140       105 2024-01-12  280.76      0.08
3           4     1219       104 2024-03-27  180.59      0.09
4           5     1125       113 2024-01-12  771.45      0.07
..        ...      ...       ...        ...     ...       ...
795       796     1031       114 2024-10-21  395.51      0.20
796       797     1119       102 2024-04-17  474.01      0.07
797       798     1045       114 2024-10-07  836.14      0.07
798       799     1143       108 2024-05-07  339.60      0.13
799       800     1195       111 2024-08-16  647.03      0.17

[780 rows x 6 columns]


In [11]:
daily_sales["year"]    = daily_sales["date"].dt.year
daily_sales["quarter"] = daily_sales["date"].dt.to_period("Q").astype(str)
orders_clean["year"]    = orders_clean["order_date"].dt.year
orders_clean["quarter"] = orders_clean["order_date"].dt.to_period("Q").astype(str)
customers["join_month"] = customers["join_date"].dt.to_period("M").astype(str)



In [13]:
reg_qtr = (daily_sales
           .groupby(["region","quarter"], as_index=False)
           .agg(revenue=("amount","sum"),
                avg_discount=("discount","mean")).round(2))

print(reg_qtr)

   region quarter    revenue  avg_discount
0      CA  2024Q1  550496.20          0.07
1      CA  2024Q2  535064.38          0.07
2      CA  2024Q3  491145.06          0.07
3      FL  2024Q1  326702.14          0.07
4      FL  2024Q2  311321.61          0.07
5      FL  2024Q3  311014.47          0.07
6      NY  2024Q1  320585.91          0.07
7      NY  2024Q2  327611.06          0.07
8      NY  2024Q3  266113.84          0.07
9      TX  2024Q1  331827.00          0.07
10     TX  2024Q2  320559.23          0.07
11     TX  2024Q3  307718.94          0.07
12     WA  2024Q1  102142.36          0.07
13     WA  2024Q2  105146.34          0.07
14     WA  2024Q3   93293.70          0.07


In [15]:
pivot_reg_qtr = reg_qtr.pivot_table(
    values="revenue", index="region", columns="quarter", aggfunc="sum"
).fillna(0)

print(pivot_reg_qtr)

quarter     2024Q1     2024Q2     2024Q3
region                                  
CA       550496.20  535064.38  491145.06
FL       326702.14  311321.61  311014.47
NY       320585.91  327611.06  266113.84
TX       331827.00  320559.23  307718.94
WA       102142.36  105146.34   93293.70


In [16]:
top_stores = (daily_sales
              .groupby(["store_id","region"], as_index=False)
              .agg(revenue=("amount","sum"),
                   avg_discount=("discount","mean"))
              .sort_values("revenue", ascending=False)
              .head(10))
print(top_stores)

    store_id region    revenue  avg_discount
11       111     TX  335993.83      0.067476
5        105     FL  329201.76      0.070714
2        102     CA  321451.55      0.070238
10       110     CA  316793.78      0.069857
12       112     NY  316296.21      0.069143
8        108     CA  314771.33      0.069143
13       113     TX  314106.26      0.068667
7        107     FL  313657.13      0.070143
9        109     CA  312687.44      0.072524
14       114     CA  311001.54      0.068333


In [17]:
orders_inner = orders_clean.merge(customers[["cust_id","region"]]
                                  .rename(columns={"region":"cust_region"}),
                                  on="cust_id", how="inner")


orders_left  = orders_clean.merge(customers[["cust_id","region"]]
                                  .rename(columns={"region":"cust_region"}),
                                  on="cust_id", how="left")


unknown_orders = orders_left["cust_region"].isna().sum()
print("Unknown-customer orders:", unknown_orders)


rev_by_cust_region = (orders_left
                      .groupby("cust_region", dropna=False, as_index=False)
                      .agg(revenue=("amount","sum")))

print(rev_by_cust_region)


Unknown-customer orders: 34
  cust_region   revenue
0          CA  75322.03
1          FL  53813.42
2          NY  52950.29
3          TX  56867.90
4          WA  34632.62
5         NaN  12875.48


In [18]:
region_totals = (daily_sales.groupby("region", as_index=False)
                 .agg(revenue=("amount","sum")))
total_rev = region_totals["revenue"].sum()
region_totals["revenue_pct"] = (region_totals["revenue"] / total_rev * 100).round(2)

print(region_totals.sort_values("revenue", ascending=False))


  region     revenue  revenue_pct
0     CA  1576705.64        33.54
3     TX   960105.17        20.42
1     FL   949038.22        20.19
2     NY   914310.81        19.45
4     WA   300582.40         6.39


In [21]:
reg_qtr["revenue_pct_in_qtr"] = (
    reg_qtr["revenue"] / reg_qtr.groupby("quarter")["revenue"].transform("sum") * 100
).round(2)


In [ ]:
print("stores dup store_id:", stores["store_id"].duplicated().sum())
print("customers dup cust_id:", customers["cust_id"].duplicated().sum())


unknown_stores = (~orders_clean["store_id"].isin(stores["store_id"])).sum()
print("Orders with unknown store_id:", unknown_stores)


print("Orders (clean):", len(orders_clean))
print("Orders with known customers (inner):", len(orders_inner))
print("Orders left join rows:", len(orders_left))

stores dup store_id: 0
customers dup cust_id: 0
Orders with unknown store_id: 0
Orders (clean): 780
Orders with known customers (inner): 746
Orders left join rows: 780
